Just defining all the parameters for the model

In [1]:
import math
import numpy as np
s = ['E', '5', 'I'] #states

ep = { 
    'E': {'A':0.25, 'C':0.25, 'G':0.25, 'T':0.25},
    '5': {'A':0.05, 'C':0, 'G':0.95, 'T':0},
    'I': {'A':0.4, 'C':0.1, 'G':0.1, 'T':0.4}
}

tp = {
    'START': {'E': 1, '5': 0, 'I': 0, 'END': 0},
    'E': {'E': 0.9, '5': 0.1, 'I': 0, 'END': 0},
    '5': {'E': 0, '5': 0, 'I': 1, 'END': 0},
    'I': {'E': 0, '5': 0, 'I': 0.9, 'END': 0.1},
    'END': {'E': 0, '5': 0, 'I': 0, 'END': 0}
}

seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
path ="EEEEEEEEEEEEEEEEEE5IIIIIII"

Part(a) - Find probability given string and states path

In [2]:
def getlogp(s, seq):
    if len(s) != len(seq):
        raise ValueError("State path and observed sequence must be of the same length.")
    
    ans = 0
    for i in range(len(s)):
        state = s[i]
        if i == 0:
            ans += math.log(tp['START'][state])
        else:
            ans += math.log(tp[s[i-1]][state])
        ans += math.log(ep[state][seq[i]])

    ans += math.log(tp[s[-1]]['END'])
    return ans

print(round(getlogp(path, seq), 3))

-41.22


IMP - To get the answer according to Sir, he is not considering the last state to end, that's why we are getting path as Es itself only.

In [3]:

dp = [[0] * len(s) for _ in range(len(seq))]
dp2 = [[0] * len(s) for _ in range(len(seq))]  

for i in range(len(s)):
    dp[0][i] = tp['START'][s[i]] * ep[s[i]][seq[0]]

for i in range(1, len(seq)):
    for j in range(len(s)):
        maxiprob = 0
        mxstate = 0
        for k in range(len(s)):
            prob = dp[i - 1][k] * tp[s[k]][s[j]] * ep[s[j]][seq[i]]
            if prob > maxiprob:
                maxiprob = prob
                mxstate = k
        dp[i][j] = maxiprob
        dp2[i][j] = mxstate

# for j in range(len(s)):
#     dp[len(seq)][j] = dp[len(seq) - 1][j] * tp[s[j]]['END']

maxiprob = 0
mxstate = 0
for j in range(len(s)):
    if dp[len(seq)-1][j] > maxiprob:
        maxiprob = dp[len(seq)-1][j]
        mxstate = j

finals = [0] * (len(seq) + 1)
finals[len(seq)] = mxstate
for i in range(len(seq) - 1, 0, -1):
    finals[i] = dp2[i][finals[i + 1]]
finals[0] = 'START'

print(f"Most probable state sequence: {[s[finals[i]] for i in range(1, len(seq) + 1)]}")
print(f"Log probability of the most probable state sequence: {round(math.log(maxiprob), 3)}")


Most probable state sequence: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']
Log probability of the most probable state sequence: -38.678


IMP - But if I consider the last state to end, then I am getting the same path as in Primer with exactly same probability.

In [4]:

dp = [[0] * len(s) for _ in range(len(seq) + 1)]
dp2 = [[0] * len(s) for _ in range(len(seq) + 1)]  

for i in range(len(s)):
    dp[0][i] = tp['START'][s[i]] * ep[s[i]][seq[0]]

for i in range(1, len(seq)):
    for j in range(len(s)):
        maxiprob = 0
        mxstate = 0
        for k in range(len(s)):
            prob = dp[i - 1][k] * tp[s[k]][s[j]] * ep[s[j]][seq[i]]
            if prob > maxiprob:
                maxiprob = prob
                mxstate = k
        dp[i][j] = maxiprob
        dp2[i][j] = mxstate

for j in range(len(s)):
    dp[len(seq)][j] = dp[len(seq) - 1][j] * tp[s[j]]['END']

maxiprob = 0
mxstate = 0
for j in range(len(s)):
    if dp[len(seq)][j] > maxiprob:
        maxiprob = dp[len(seq)][j]
        mxstate = j

finals = [0] * (len(seq) + 1)
finals[len(seq)] = mxstate
for i in range(len(seq) - 1, 0, -1):
    finals[i] = dp2[i][finals[i + 1]]
finals[0] = 'START'


print(f"Most probable state sequence: {[s[finals[i]] for i in range(1, len(seq) + 1)]}")
print(f"Log probability of the given path: {math.log(maxiprob, 2.718)}")


Most probable state sequence: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '5', 'I', 'I', 'I', 'I', 'I', 'I', 'I']
Log probability of the given path: -41.22395196143138
